**Importing the dataset form kaggle...**

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d yasserh/housing-prices-dataset

Dataset URL: https://www.kaggle.com/datasets/yasserh/housing-prices-dataset
License(s): CC0-1.0
  0% 0.00/4.63k [00:00<?, ?B/s]
100% 4.63k/4.63k [00:00<00:00, 10.2MB/s]


In [4]:
from zipfile import ZipFile
file_name = "housing-prices-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [64]:
data = pd.read_csv("Housing.csv")
x = data.iloc[:, 1:13].values
y = data.iloc[:, 0].values

In [42]:
data.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [65]:
# Doing one hot encodinng for the last column....
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [11])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [67]:
# Doing the encoding wala thing....
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(7,12):
  x[:, i] = le.fit_transform(x[:, i])
x[:,13] = le.fit_transform(x[:,13])

[[1.0 0.0 0.0 ... 1 2 1]
 [1.0 0.0 0.0 ... 1 3 0]
 [0.0 1.0 0.0 ... 0 2 1]
 ...
 [0.0 0.0 1.0 ... 0 0 0]
 [1.0 0.0 0.0 ... 0 0 0]
 [0.0 0.0 1.0 ... 0 0 0]]


In [68]:
#Splitting the data....
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

**Feature scaling for svr...**

In [80]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
x_scale_train = sc_X.fit_transform(x_train)
y_scale_train = sc_y.fit_transform(y_train.reshape(-1,1))

#Training the Model.....#

**Using polynomial regression...**

In [71]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 4)
x_poly = poly_reg.fit_transform(x_train)
poly_reg_model = LinearRegression()
poly_reg_model.fit(x_poly, y_train)

LinearRegression()

**Using Multiple Linear Regerssion...**

In [70]:
from sklearn.linear_model import LinearRegression
mult_reg = LinearRegression()
mult_reg.fit(x_train, y_train)

LinearRegression()

**Using Decision Tree...**

In [72]:
from sklearn.tree import DecisionTreeRegressor
dec_tree_reg = DecisionTreeRegressor()
dec_tree_reg.fit(x_train, y_train)

DecisionTreeRegressor()

**Random Forest Reg....**

In [101]:
from sklearn.ensemble import RandomForestRegressor
rand_for_reg = RandomForestRegressor(n_estimators = 100)
rand_for_reg.fit(x_train, y_train)

RandomForestRegressor()

In [81]:
from sklearn.svm import SVR
svr_reg = SVR(kernel = 'rbf')
svr_reg.fit(x_scale_train, y_scale_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [102]:
##Now predict using all reg models....
# ans1 = poly_reg_model.predict(x_train)
ans2 = mult_reg.predict(x_train)
ans3 = dec_tree_reg.predict(x_train)
ans4 = rand_for_reg.predict(x_train)
ans5 = sc_y.inverse_transform((svr_reg.predict(sc_X.transform(x_train))).reshape(-1,1))

In [103]:
##predicting r2 score...
from sklearn.metrics import r2_score
s1 = r2_score(y_train, ans2)
s2 = r2_score(y_train, ans3)
s3 = r2_score(y_train, ans4)
s4 = r2_score(y_train, ans5)
print(s1,s2,s3,s4)



0.6813634599707064 0.9998179006928982 0.9467110306379695 0.7930447145165302


**Now testing on test data....**

In [104]:
##Now predict using all reg models....
ans6 = mult_reg.predict(x_test)
ans7 = dec_tree_reg.predict(x_test)
ans8 = rand_for_reg.predict(x_test)
ans9 = sc_y.inverse_transform((svr_reg.predict(sc_X.transform(x_test))).reshape(-1,1))

In [105]:
from sklearn.metrics import r2_score
s5 = r2_score(y_test, ans6)
s6 = r2_score(y_test, ans7)
s7 = r2_score(y_test, ans8)
s8 = r2_score(y_test, ans9)
print(s5,s6,s7,s8)


0.6645127695906055 0.4669657927633132 0.6786897777935839 0.6283304039983


In [100]:
print(x[:,:15])

[[1.0 0.0 0.0 ... 1 2 1]
 [1.0 0.0 0.0 ... 1 3 0]
 [0.0 1.0 0.0 ... 0 2 1]
 ...
 [0.0 0.0 1.0 ... 0 0 0]
 [1.0 0.0 0.0 ... 0 0 0]
 [0.0 0.0 1.0 ... 0 0 0]]
